In [2]:
%pip install langchain chromadb pypdf sentence_transformers InstructorEmbedding streamlit bitsandbytes ctransformers[cuda]

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [1]:
import torch
print(torch.cuda.is_available())

True


In [3]:
import sys
sys.executable

'c:\\Python311\\python.exe'

In [4]:
# New Backend Imports

import torch
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.llms import LlamaCpp

from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings


c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Old backend imports

import git
import os
import requests

from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import DeepLake
from langchain.chains import RetrievalQA
from langchain.llms.huggingface_pipeline import HuggingFacePipeline

from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering
from transformers import pipeline
import torch

In [5]:
from langchain.llms import CTransformers

config = {'max_new_tokens': 2048, 'temperature': 0.1, 'context_length': 2048}
llm = CTransformers(model='TheBloke/Mistral-7B-Instruct-v0.1-GGUF',model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf", config=config, n_ctx=4096)

Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 317.87it/s]


In [6]:
# Load and process the text files

loader=PyPDFLoader('data/Resume_balanced.pdf')
# loader = DirectoryLoader('data', glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()

In [6]:
# #splitting the text into
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=300)
# texts = text_splitter.split_documents(documents)

In [7]:
# len(texts)

4

In [8]:
# instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large",
#                                                       model_kwargs={'device': 'cuda:0'})

load INSTRUCTOR_Transformer
max_seq_length  512


In [9]:
# # Embed and store the texts
# # Supplying a persist_directory will store the embeddings on disk
# persist_directory = 'db'

# ## Here is the nmew embeddings being used
# embedding = instructor_embeddings

# # vectordb = Chroma.from_documents(documents=texts,
# #                                  embedding=embedding,
# #                                  persist_directory=persist_directory)

# vectordb = Chroma.from_documents(documents=texts,
#                                  embedding=embedding,)

# # persiste the db to disk
# # vectordb.persist()
# # vectordb = None

AssertionError: Torch not compiled with CUDA enabled

In [15]:
# retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [16]:
# create the chain to answer questions
# qa_chain = RetrievalQA.from_chain_type(llm=llm,
#                                   chain_type="stuff",
#                                   retriever=retriever,
#                                   return_source_documents=True)

In [17]:
# ## Cite sources

# import textwrap

# def wrap_text_preserve_newlines(text, width=110):
#     # Split the input text into lines based on newline characters
#     lines = text.split('\n')

#     # Wrap each line individually
#     wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

#     # Join the wrapped lines back together using newline characters
#     wrapped_text = '\n'.join(wrapped_lines)

#     return wrapped_text

# def process_llm_response(llm_response):
#     print(wrap_text_preserve_newlines(llm_response['result']))
#     print('\n\nSources:')
#     for source in llm_response["source_documents"]:
#         print(source.metadata['source'])

In [8]:
def processGitLink(git_link) -> None:
    last_name = git_link.split("/")[-1]
    clone_path = last_name.split(".")[0]
    return clone_path

In [9]:
def clone_repo(git_link, clone_path):
    if not os.path.exists(clone_path):
        git.Repo.clone_from(git_link, clone_path)
        return


def extract_all_files(clone_path, docs, allowed_extensions):
    root_dir = clone_path
    for dirpath, _, filenames in os.walk(root_dir):
        for file in filenames:
            file_extension = os.path.splitext(file)[1]
            if file_extension in allowed_extensions:
                try:
                    loader = TextLoader(os.path.join(dirpath, file), encoding="utf-8")
                    docs.extend(loader.load_and_split())
                except Exception as e:
                    pass
    return


In [10]:
def chunk_files(docs):
    # text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    # texts = text_splitter.split_documents(docs)
    #splitting the text into
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    return texts


# def embed_deeplake(texts, deeplake_path, hf):
#     db = DeepLake(dataset_path=deeplake_path, embedding_function=hf, overwrite=True)
#     db.add_documents(texts)
#     return db

In [11]:
texts=chunk_files(documents)

In [12]:
len(texts)

3

In [13]:
def create_vectordb(texts):
    # db = DeepLake(dataset_path=deeplake_path, embedding_function=hf, overwrite=True)
    # db.add_documents(texts)
    # return db

    instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large",
                                                      model_kwargs={'device': 'cuda:0'})
    # Embed and store the texts
    # Supplying a persist_directory will store the embeddings on disk
    # persist_directory = 'db'

    ## Here is the nmew embeddings being used
    embedding = instructor_embeddings

    # vectordb = Chroma.from_documents(documents=texts,
    #                                  embedding=embedding,
    #                                  persist_directory=persist_directory)

    vectordb = Chroma.from_documents(documents=texts,
                                    embedding=embedding,)
    
    return vectordb

    # persiste the db to disk
    # vectordb.persist()
    # vectordb = None

In [14]:
vectordb=create_vectordb(texts)

load INSTRUCTOR_Transformer
max_seq_length  512


In [15]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)


In [16]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [17]:
%%time

# example
query = "What are the strengths of candidate, Aditya?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Number of tokens (1025) exceeded maximum context length (1024).
Number of tokens (1026) exceeded maximum context length (1024).
Number of tokens (1027) exceeded maximum context length (1024).
Number of tokens (1028) exceeded maximum context length (1024).
Number of tokens (1029) exceeded maximum context length (1024).
Number of tokens (1030) exceeded maximum context length (1024).
Number of tokens (1031) exceeded maximum context length (1024).
Number of tokens (1032) exceeded maximum context length (1024).
Number of tokens (1033) exceeded maximum context length (1024).
Number of tokens (1034) exceeded maximum context length (1024).
Number of tokens (1035) exceeded maximum context length (1024).
Number of tokens (1036) exceeded maximum context length (1024).
Number of tokens (1037) exceeded maximum context length (1024).
Number of tokens (1038) exceeded maximum context length (1024).
Number of tokens (1039) exceeded maximum context length (1024).
Number of tokens (1040) exceeded maximum

 Aditya has several strengths as a candidate. He is a state topper in CBSE Class 10th exams and an NTSE
Scholar, indicating his academic excellence. He has also received a prestigious fellowship from IBITF for his
work on the Divyadrishti mobile application. In addition to his academic achievements, Aditya has experience
working as a Project Assistant at Magnetica, Australia, where he was involved in R&D and C++ implementation of
an algorithm to illuminate low signal regions of MRI regions of MRI regions of MRI regions of MRI regions of
MRI regions of MRI regions of MRI regions of MRI regions of MRI regions of MRI regions of MRI regions of MRI
regions of MRI regions of MRI regions of MRI regions of MRI regions of MRI regions of MRI regions of MRI
regions of MRI regions of MRI regions of MRI regions of MRI regions of MRI regions of MRI regions of MRI
regions of MRI regions of MRI regions of MRI regions of MRI regions of MRI regions of MRI regions of MRI
regions of MRI regions of MRI reg